In [5]:
import xgboost as xgb
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn import model_selection, preprocessing, ensemble
from sklearn.metrics import explained_variance_score
import xgboost as xgb
import numpy as np
import pandas as pd
import lightgbm as lgb
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV  

In [18]:
# coding=utf8
import numpy as np
import pandas as pd
import lightgbm as lgb
import matplotlib.pyplot as plt

RS = 20170501
np.random.seed(RS)

ROUNDS = 450
params = {
	'objective': 'regression',
    'metric': 'rmse',
    'boosting': 'gbdt',
    'learning_rate': 0.04,
    'verbose': 0,
    'num_leaves': 2 ** 5,
    'bagging_fraction': 0.95,
    'bagging_freq': 1,
    'bagging_seed': RS,
    'feature_fraction': 0.7,
    'feature_fraction_seed': RS,
    'max_bin': 100,
    'max_depth': 5,
    'num_rounds': ROUNDS
}

print("Started")
input_folder = ''
train_df = pd.read_csv(input_folder + 'train.csv', parse_dates=['timestamp'])
test_df  = pd.read_csv(input_folder + 'test.csv' , parse_dates=['timestamp'])
macro_df = pd.read_csv(input_folder + 'macro.csv', parse_dates=['timestamp'])

#fix outlier
train_df.drop(train_df[train_df["life_sq"] > 5000].index, inplace=True)

train_y  = np.log(train_df['price_doc'].values)
test_ids = test_df['id']

train_df.drop(['id', 'price_doc'], axis=1, inplace=True)
test_df.drop(['id'], axis=1, inplace=True)
print("Data: X_train: {}, X_test: {}".format(train_df.shape, test_df.shape))

df = pd.concat([train_df, test_df])

#Lets try using only those from https://www.kaggle.com/robertoruiz/sberbank-russian-housing-market/dealing-with-multicollinearity
macro_cols = ["timestamp","balance_trade","balance_trade_growth","eurrub","average_provision_of_build_contract","micex_rgbi_tr","micex_cbi_tr","deposits_rate","mortgage_value","mortgage_rate","income_per_cap","museum_visitis_per_100_cap","apartment_build"]
df = df.merge(macro_df[macro_cols], on='timestamp', how='left')
print("Merged with macro: {}".format(df.shape))

#Dates...
df['year'] = df.timestamp.dt.year
df['month'] = df.timestamp.dt.month
df['dow'] = df.timestamp.dt.dayofweek
df.drop(['timestamp'], axis=1, inplace=True)

#More featuers needed...

df_num = df.select_dtypes(exclude=['object'])
df_obj = df.select_dtypes(include=['object']).copy()
for c in df_obj:
    df_obj[c] = pd.factorize(df_obj[c])[0]

df_values = pd.concat([df_num, df_obj], axis=1)

pos = train_df.shape[0]
train_df = df_values[:pos]
test_df  = df_values[pos:]
del df, df_num, df_obj, df_values

print("Training on: {}".format(train_df.shape, train_y.shape))

train_lgb = lgb.Dataset(train_df, train_y)
model = lgb.train(params, train_lgb, num_boost_round=ROUNDS)
preds = model.predict(test_df)
	
print("Writing output...")
out_df = pd.DataFrame({"id":test_ids, "price_doc":np.exp(preds)})
out_df.to_csv("lgb_{}_{}.csv".format(ROUNDS, RS), index=False)
print(out_df.head(3))

print("Features importance...")
gain = model.feature_importance('gain')
ft = pd.DataFrame({'feature':model.feature_name(), 'split':model.feature_importance('split'), 'gain':100 * gain / gain.sum()}).sort_values('gain', ascending=False)
print(ft.head(25))

plt.figure()
ft[['feature','gain']].head(25).plot(kind='barh', x='feature', y='gain', legend=False, figsize=(10, 20))
plt.gcf().savefig('features_importance.png')

print("Done.")

Started
Data: X_train: (30470, 290), X_test: (7662, 290)
Merged with macro: (38132, 302)
Training on: (30470, 304)
Writing output...
      id     price_doc
0  30474  5.543121e+06
1  30475  8.184943e+06
2  30476  5.362328e+06
Features importance...
                        feature       gain  split
0                       full_sq  30.750326    524
1                       life_sq   6.107728    205
6                      num_room   5.709228     69
211             cafe_count_2000   3.289324     46
267  cafe_count_5000_price_high   2.677927     44
265  cafe_count_5000_price_2500   2.487797     33
249            sport_count_3000   1.932199     33
234             cafe_count_3000   1.631737     21
5                    build_year   1.387829    236
72               metro_min_avto   1.103576     54
219  cafe_count_2000_price_2500   1.043807     18
279                micex_cbi_tr   1.018998    151
7                      kitch_sq   0.947734     90
2                         floor   0.854271    191
27

In [3]:
train_y

array([ 15.58195222,  15.60727003,  15.55597673, ...,  15.75726336,
        16.41820024,  15.53827716])

In [1]:
# thank's to https://www.kaggle.com/aharless for code sharing
# https://www.kaggle.com/aharless/jiwon-small-improvements-for-magic-number-results/versions

## This is a straight forward fork but I added some extra features that I found pretty useful.
## The featues with a single grid search XGBoost Model got me to around a 0.31780.
## I couldn't get any of my other models into the ball park that this one is at and I think it
## is a pretty neat combination!

## 6.27.17
## It was pointed out (in the comments) that I had a double instance of a feature. Oddly, the score 
## is better with both of them in. I havent figured out why yet.

# Parameters
prediction_stderr = 0.0073  #  assumed standard error of predictions
                          #  (smaller values make output closer to input)
train_test_logmean_diff = 0.1  # assumed shift used to adjust frequencies for time trend
probthresh = 90  # minimum probability*frequency to use new price instead of just rounding
rounder = 2  # number of places left of decimal point to zero


import numpy as np
import pandas as pd
from sklearn import model_selection, preprocessing
import xgboost as xgb

import datetime
from scipy.stats import norm
    
#load files
origin_train = pd.read_csv('train.csv', parse_dates=['timestamp'])
origin_test = pd.read_csv('test.csv', parse_dates=['timestamp'])
train = pd.read_csv('train.csv', parse_dates=['timestamp'])
test = pd.read_csv('test.csv', parse_dates=['timestamp'])
id_test = test.id

print(train.shape)

#clean data
print('Data Clean...')
bad_index = train[train.life_sq > train.full_sq].index
train.loc[bad_index, "life_sq"] = np.NaN
equal_index = [601,1896,2791]
test.loc[equal_index, "life_sq"] = test.loc[equal_index, "full_sq"]
bad_index = test[test.life_sq > test.full_sq].index
test.loc[bad_index, "life_sq"] = np.NaN
bad_index = train[train.life_sq < 5].index
train.loc[bad_index, "life_sq"] = np.NaN
bad_index = test[test.life_sq < 5].index
test.loc[bad_index, "life_sq"] = np.NaN
bad_index = train[train.full_sq < 5].index
train.loc[bad_index, "full_sq"] = np.NaN
bad_index = test[test.full_sq < 5].index
test.loc[bad_index, "full_sq"] = np.NaN
kitch_is_build_year = [13117]
train.loc[kitch_is_build_year, "build_year"] = train.loc[kitch_is_build_year, "kitch_sq"]
bad_index = train[train.kitch_sq >= train.life_sq].index
train.loc[bad_index, "kitch_sq"] = np.NaN
bad_index = test[test.kitch_sq >= test.life_sq].index
test.loc[bad_index, "kitch_sq"] = np.NaN
bad_index = train[(train.kitch_sq == 0).values + (train.kitch_sq == 1).values].index
train.loc[bad_index, "kitch_sq"] = np.NaN
bad_index = test[(test.kitch_sq == 0).values + (test.kitch_sq == 1).values].index
test.loc[bad_index, "kitch_sq"] = np.NaN
bad_index = train[(train.full_sq > 210) & (train.life_sq / train.full_sq < 0.3)].index
train.loc[bad_index, "full_sq"] = np.NaN
bad_index = test[(test.full_sq > 150) & (test.life_sq / test.full_sq < 0.3)].index
test.loc[bad_index, "full_sq"] = np.NaN
bad_index = train[train.life_sq > 300].index
train.loc[bad_index, ["life_sq", "full_sq"]] = np.NaN
bad_index = test[test.life_sq > 200].index
test.loc[bad_index, ["life_sq", "full_sq"]] = np.NaN
train.product_type.value_counts(normalize= True)
test.product_type.value_counts(normalize= True)
bad_index = train[train.build_year < 1500].index
train.loc[bad_index, "build_year"] = np.NaN
bad_index = test[test.build_year < 1500].index
test.loc[bad_index, "build_year"] = np.NaN
bad_index = train[train.num_room == 0].index
train.loc[bad_index, "num_room"] = np.NaN
bad_index = test[test.num_room == 0].index
test.loc[bad_index, "num_room"] = np.NaN
bad_index = [10076, 11621, 17764, 19390, 24007, 26713, 29172]
train.loc[bad_index, "num_room"] = np.NaN
bad_index = [3174, 7313]
test.loc[bad_index, "num_room"] = np.NaN
bad_index = train[(train.floor == 0).values * (train.max_floor == 0).values].index
train.loc[bad_index, ["max_floor", "floor"]] = np.NaN
bad_index = train[train.floor == 0].index
train.loc[bad_index, "floor"] = np.NaN
bad_index = train[train.max_floor == 0].index
train.loc[bad_index, "max_floor"] = np.NaN
bad_index = test[test.max_floor == 0].index
test.loc[bad_index, "max_floor"] = np.NaN
bad_index = train[train.floor > train.max_floor].index
train.loc[bad_index, "max_floor"] = np.NaN
bad_index = test[test.floor > test.max_floor].index
test.loc[bad_index, "max_floor"] = np.NaN
train.floor.describe(percentiles= [0.9999])
bad_index = [23584]
train.loc[bad_index, "floor"] = np.NaN
train.material.value_counts()
test.material.value_counts()
train.state.value_counts()
bad_index = train[train.state == 33].index
train.loc[bad_index, "state"] = np.NaN
test.state.value_counts()

print(train.shape)

# brings error down a lot by removing extreme price per sqm
train.loc[train.full_sq == 0, 'full_sq'] = 50
train = train[train.price_doc/train.full_sq <= 600000]
train = train[train.price_doc/train.full_sq >= 10000]

print(train.shape)

print('Feature Engineering...')
# Add month-year
month_year = (train.timestamp.dt.month*30 + train.timestamp.dt.year * 365)
month_year_cnt_map = month_year.value_counts().to_dict()
train['month_year_cnt'] = month_year.map(month_year_cnt_map)

month_year = (test.timestamp.dt.month*30 + test.timestamp.dt.year * 365)
month_year_cnt_map = month_year.value_counts().to_dict()
test['month_year_cnt'] = month_year.map(month_year_cnt_map)

# Add week-year count
week_year = (train.timestamp.dt.weekofyear*7 + train.timestamp.dt.year * 365)
week_year_cnt_map = week_year.value_counts().to_dict()
train['week_year_cnt'] = week_year.map(week_year_cnt_map)

week_year = (test.timestamp.dt.weekofyear*7 + test.timestamp.dt.year * 365)
week_year_cnt_map = week_year.value_counts().to_dict()
test['week_year_cnt'] = week_year.map(week_year_cnt_map)

# Add month and day-of-week
train['month'] = train.timestamp.dt.month
train['dow'] = train.timestamp.dt.dayofweek

test['month'] = test.timestamp.dt.month
test['dow'] = test.timestamp.dt.dayofweek

# Other feature engineering
train['rel_floor'] = 0.05+train['floor'] / train['max_floor'].astype(float)
train['rel_kitch_sq'] = 0.05+train['kitch_sq'] / train['full_sq'].astype(float)

test['rel_floor'] = 0.05+test['floor'] / test['max_floor'].astype(float)
test['rel_kitch_sq'] = 0.05+test['kitch_sq'] / test['full_sq'].astype(float)

train.apartment_name=train.sub_area + train['metro_km_avto'].astype(str)
test.apartment_name=test.sub_area + train['metro_km_avto'].astype(str)

train['room_size'] = train['life_sq'] / train['num_room'].astype(float)
test['room_size'] = test['life_sq'] / test['num_room'].astype(float)

train['area_per_room'] = train['life_sq'] / train['num_room'].astype(float) #rough area per room
train['livArea_ratio'] = train['life_sq'] / train['full_sq'].astype(float) #rough living area
train['yrs_old'] = 2017 - train['build_year'].astype(float) #years old from 2017
train['avgfloor_sq'] = train['life_sq']/train['max_floor'].astype(float) #living area per floor
train['pts_floor_ratio'] = train['public_transport_station_km']/train['max_floor'].astype(float)
# looking for significance of apartment buildings near public t 
train['room_size'] = train['life_sq'] / train['num_room'].astype(float)
# doubled a var by accident
# when removing one score did not improve...
train['gender_ratio'] = train['male_f']/train['female_f'].astype(float)
train['kg_park_ratio'] = train['kindergarten_km']/train['park_km'].astype(float) #significance of children?
train['high_ed_extent'] = train['school_km'] / train['kindergarten_km'] #schooling
train['pts_x_state'] = train['public_transport_station_km'] * train['state'].astype(float) #public trans * state of listing
train['lifesq_x_state'] = train['life_sq'] * train['state'].astype(float) #life_sq times the state of the place
train['floor_x_state'] = train['floor'] * train['state'].astype(float) #relative floor * the state of the place

test['area_per_room'] = test['life_sq'] / test['num_room'].astype(float)
test['livArea_ratio'] = test['life_sq'] / test['full_sq'].astype(float)
test['yrs_old'] = 2017 - test['build_year'].astype(float)
test['avgfloor_sq'] = test['life_sq']/test['max_floor'].astype(float) #living area per floor
test['pts_floor_ratio'] = test['public_transport_station_km']/test['max_floor'].astype(float) #apartments near public t?
test['room_size'] = test['life_sq'] / test['num_room'].astype(float)
test['gender_ratio'] = test['male_f']/test['female_f'].astype(float)
test['kg_park_ratio'] = test['kindergarten_km']/test['park_km'].astype(float)
test['high_ed_extent'] = test['school_km'] / test['kindergarten_km']
test['pts_x_state'] = test['public_transport_station_km'] * test['state'].astype(float) #public trans * state of listing
test['lifesq_x_state'] = test['life_sq'] * test['state'].astype(float)
test['floor_x_state'] = test['floor'] * test['state'].astype(float)

print(train.shape)
#########################################################################
print('Rate Mults...')
# Aggreagte house price data derived from 
# http://www.globalpropertyguide.com/real-estate-house-prices/R#russia
# by luckyzhou
# See https://www.kaggle.com/luckyzhou/lzhou-test/comments

rate_2015_q2 = 1
rate_2015_q1 = rate_2015_q2 / 0.9932
rate_2014_q4 = rate_2015_q1 / 1.0112
rate_2014_q3 = rate_2014_q4 / 1.0169
rate_2014_q2 = rate_2014_q3 / 1.0086
rate_2014_q1 = rate_2014_q2 / 1.0126
rate_2013_q4 = rate_2014_q1 / 0.9902
rate_2013_q3 = rate_2013_q4 / 1.0041
rate_2013_q2 = rate_2013_q3 / 1.0044
rate_2013_q1 = rate_2013_q2 / 1.0104  # This is 1.002 (relative to mult), close to 1:
rate_2012_q4 = rate_2013_q1 / 0.9832  #     maybe use 2013q1 as a base quarter and get rid of mult?
rate_2012_q3 = rate_2012_q4 / 1.0277
rate_2012_q2 = rate_2012_q3 / 1.0279
rate_2012_q1 = rate_2012_q2 / 1.0279
rate_2011_q4 = rate_2012_q1 / 1.076
rate_2011_q3 = rate_2011_q4 / 1.0236
rate_2011_q2 = rate_2011_q3 / 1
rate_2011_q1 = rate_2011_q2 / 1.011

# train 2015
train['average_q_price'] = 1

train_2015_q2_index = train.loc[train['timestamp'].dt.year == 2015].loc[train['timestamp'].dt.month >= 4].loc[train['timestamp'].dt.month < 7].index
train.loc[train_2015_q2_index, 'average_q_price'] = rate_2015_q2

train_2015_q1_index = train.loc[train['timestamp'].dt.year == 2015].loc[train['timestamp'].dt.month >= 1].loc[train['timestamp'].dt.month < 4].index
train.loc[train_2015_q1_index, 'average_q_price'] = rate_2015_q1


# train 2014
train_2014_q4_index = train.loc[train['timestamp'].dt.year == 2014].loc[train['timestamp'].dt.month >= 10].loc[train['timestamp'].dt.month <= 12].index
train.loc[train_2014_q4_index, 'average_q_price'] = rate_2014_q4

train_2014_q3_index = train.loc[train['timestamp'].dt.year == 2014].loc[train['timestamp'].dt.month >= 7].loc[train['timestamp'].dt.month < 10].index
train.loc[train_2014_q3_index, 'average_q_price'] = rate_2014_q3

train_2014_q2_index = train.loc[train['timestamp'].dt.year == 2014].loc[train['timestamp'].dt.month >= 4].loc[train['timestamp'].dt.month < 7].index
train.loc[train_2014_q2_index, 'average_q_price'] = rate_2014_q2

train_2014_q1_index = train.loc[train['timestamp'].dt.year == 2014].loc[train['timestamp'].dt.month >= 1].loc[train['timestamp'].dt.month < 4].index
train.loc[train_2014_q1_index, 'average_q_price'] = rate_2014_q1


# train 2013
train_2013_q4_index = train.loc[train['timestamp'].dt.year == 2013].loc[train['timestamp'].dt.month >= 10].loc[train['timestamp'].dt.month <= 12].index
train.loc[train_2013_q4_index, 'average_q_price'] = rate_2013_q4

train_2013_q3_index = train.loc[train['timestamp'].dt.year == 2013].loc[train['timestamp'].dt.month >= 7].loc[train['timestamp'].dt.month < 10].index
train.loc[train_2013_q3_index, 'average_q_price'] = rate_2013_q3

train_2013_q2_index = train.loc[train['timestamp'].dt.year == 2013].loc[train['timestamp'].dt.month >= 4].loc[train['timestamp'].dt.month < 7].index
train.loc[train_2013_q2_index, 'average_q_price'] = rate_2013_q2

train_2013_q1_index = train.loc[train['timestamp'].dt.year == 2013].loc[train['timestamp'].dt.month >= 1].loc[train['timestamp'].dt.month < 4].index
train.loc[train_2013_q1_index, 'average_q_price'] = rate_2013_q1


# train 2012
train_2012_q4_index = train.loc[train['timestamp'].dt.year == 2012].loc[train['timestamp'].dt.month >= 10].loc[train['timestamp'].dt.month <= 12].index
train.loc[train_2012_q4_index, 'average_q_price'] = rate_2012_q4

train_2012_q3_index = train.loc[train['timestamp'].dt.year == 2012].loc[train['timestamp'].dt.month >= 7].loc[train['timestamp'].dt.month < 10].index
train.loc[train_2012_q3_index, 'average_q_price'] = rate_2012_q3

train_2012_q2_index = train.loc[train['timestamp'].dt.year == 2012].loc[train['timestamp'].dt.month >= 4].loc[train['timestamp'].dt.month < 7].index
train.loc[train_2012_q2_index, 'average_q_price'] = rate_2012_q2

train_2012_q1_index = train.loc[train['timestamp'].dt.year == 2012].loc[train['timestamp'].dt.month >= 1].loc[train['timestamp'].dt.month < 4].index
train.loc[train_2012_q1_index, 'average_q_price'] = rate_2012_q1


# train 2011
train_2011_q4_index = train.loc[train['timestamp'].dt.year == 2011].loc[train['timestamp'].dt.month >= 10].loc[train['timestamp'].dt.month <= 12].index
train.loc[train_2011_q4_index, 'average_q_price'] = rate_2011_q4

train_2011_q3_index = train.loc[train['timestamp'].dt.year == 2011].loc[train['timestamp'].dt.month >= 7].loc[train['timestamp'].dt.month < 10].index
train.loc[train_2011_q3_index, 'average_q_price'] = rate_2011_q3

train_2011_q2_index = train.loc[train['timestamp'].dt.year == 2011].loc[train['timestamp'].dt.month >= 4].loc[train['timestamp'].dt.month < 7].index
train.loc[train_2011_q2_index, 'average_q_price'] = rate_2011_q2

train_2011_q1_index = train.loc[train['timestamp'].dt.year == 2011].loc[train['timestamp'].dt.month >= 1].loc[train['timestamp'].dt.month < 4].index
train.loc[train_2011_q1_index, 'average_q_price'] = rate_2011_q1

train['price_doc'] = train['price_doc'] * train['average_q_price']


#########################################################################################################

mult = 1.054880504
train['price_doc'] = train['price_doc'] * mult
y_train = train["price_doc"]
pd.set_option('max_rows',40)
train.describe().astype(np.int64).T
print(train.shape)

c:\python27\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


(30471, 292)
Data Clean...
(30471, 292)
(30396, 292)
Feature Engineering...
(30396, 310)
Rate Mults...
(30396, 311)


In [2]:
print('Running Model 3...')
df_train = pd.read_csv("train.csv", parse_dates=['timestamp'])
df_test = pd.read_csv("test.csv", parse_dates=['timestamp'])
df_macro = pd.read_csv("macro.csv", parse_dates=['timestamp'])

df_train.drop(df_train[df_train["life_sq"] > 7000].index, inplace=True)

mult = 0.969
y_train = df_train['price_doc'].values * mult + 10
id_test = df_test['id']
id_train = df_train['id']

remove_index_3 = (set(origin_train['id']).difference(set(df_train['id']))) 

df_train.drop(['id', 'price_doc'], axis=1, inplace=True)
df_test.drop(['id'], axis=1, inplace=True)

num_train = len(df_train)
df_all = pd.concat([df_train, df_test])
# Next line just adds a lot of NA columns (becuase "join" only works on indexes)
# but somewhow it seems to affect the result
df_all = df_all.join(df_macro, on='timestamp', rsuffix='_macro')
print(df_all.shape)

# Add month-year
month_year = (df_all.timestamp.dt.month*30 + df_all.timestamp.dt.year * 365)
month_year_cnt_map = month_year.value_counts().to_dict()
df_all['month_year_cnt'] = month_year.map(month_year_cnt_map)

# Add week-year count
week_year = (df_all.timestamp.dt.weekofyear*7 + df_all.timestamp.dt.year * 365)
week_year_cnt_map = week_year.value_counts().to_dict()
df_all['week_year_cnt'] = week_year.map(week_year_cnt_map)

# Add month and day-of-week
df_all['month'] = df_all.timestamp.dt.month
df_all['dow'] = df_all.timestamp.dt.dayofweek

# Other feature engineering
df_all['rel_floor'] = df_all['floor'] / df_all['max_floor'].astype(float)
df_all['rel_kitch_sq'] = df_all['kitch_sq'] / df_all['full_sq'].astype(float)

## same ones as above 
df_all['area_per_room'] = df_all['life_sq'] / df_all['num_room'].astype(float)
df_all['livArea_ratio'] = df_all['life_sq'] / df_all['full_sq'].astype(float)
df_all['yrs_old'] = 2017 - df_all['build_year'].astype(float)
df_all['avgfloor_sq'] = df_all['life_sq']/df_all['max_floor'].astype(float) #living area per floor
df_all['pts_floor_ratio'] = df_all['public_transport_station_km']/df_all['max_floor'].astype(float) #apartments near public t?
df_all['room_size'] = df_all['life_sq'] / df_all['num_room'].astype(float)
df_all['gender_ratio'] = df_all['male_f']/df_all['female_f'].astype(float)
df_all['kg_park_ratio'] = df_all['kindergarten_km']/df_all['park_km'].astype(float)
df_all['high_ed_extent'] = df_all['school_km'] / df_all['kindergarten_km']
df_all['pts_x_state'] = df_all['public_transport_station_km'] * df_all['state'].astype(float) #public trans * state of listing
df_all['lifesq_x_state'] = df_all['life_sq'] * df_all['state'].astype(float)
df_all['floor_x_state'] = df_all['floor'] * df_all['state'].astype(float)

##macro feature adds
df_all['micex_cbi_ratio'] = df_all['micex_cbi_tr']/df_all['micex'].astype(float)
df_all['micex_rgbi_ratio'] = df_all['micex_rgbi_tr']/df_all['micex'].astype(float)

train['building_name'] = pd.factorize(train.sub_area + train['metro_km_avto'].astype(str))[0]
test['building_name'] = pd.factorize(test.sub_area + test['metro_km_avto'].astype(str))[0]

def add_time_features(col):
   col_month_year = pd.Series(pd.factorize(train[col].astype(str) + month_year.astype(str))[0])
   train[col + '_month_year_cnt'] = col_month_year.map(col_month_year.value_counts())

   col_week_year = pd.Series(pd.factorize(train[col].astype(str) + week_year.astype(str))[0])
   train[col + '_week_year_cnt'] = col_week_year.map(col_week_year.value_counts())

add_time_features('building_name')
add_time_features('sub_area')

def add_time_features(col):
   col_month_year = pd.Series(pd.factorize(test[col].astype(str) + month_year.astype(str))[0])
   test[col + '_month_year_cnt'] = col_month_year.map(col_month_year.value_counts())

   col_week_year = pd.Series(pd.factorize(test[col].astype(str) + week_year.astype(str))[0])
   test[col + '_week_year_cnt'] = col_week_year.map(col_week_year.value_counts())

add_time_features('building_name')
add_time_features('sub_area')


# Remove timestamp column (may overfit the model in train)
df_all.drop(['timestamp', 'timestamp_macro'], axis=1, inplace=True)


factorize = lambda t: pd.factorize(t[1])[0]

df_obj = df_all.select_dtypes(include=['object'])

X_all = np.c_[
    df_all.select_dtypes(exclude=['object']).values,
    np.array(list(map(factorize, df_obj.iteritems()))).T
]
print(X_all.shape)

X_train = X_all[:num_train]
X_test = X_all[num_train:]


# Deal with categorical values
df_numeric = df_all.select_dtypes(exclude=['object'])
df_obj = df_all.select_dtypes(include=['object']).copy()

for c in df_obj:
    df_obj[c] = pd.factorize(df_obj[c])[0]

df_values = pd.concat([df_numeric, df_obj], axis=1)


# Convert to numpy values
X_all = df_values.values
print(X_all.shape)

train_df = df_values[:num_train]
train_y = np.log(y_train)

Running Model 3...
(38132, 390)
(38132L, 408L)
(38132L, 408L)


In [9]:
train_y[0]

15.550463316203563

In [17]:
params = { 
        'objective': 'regression',
        'metric': 'rmse',
        'boosting': 'gbdt',
        'learning_rate': 0.05,
        'verbose': 0,
        'num_leaves': 2 ** 5,
        'bagging_fraction': 0.95,
        'bagging_freq': 1,
        'feature_fraction': 0.7,
        'max_bin': 100,
        'max_depth': 6
    }  

features = df_values.describe().T.index
train_df = df_values[features].as_Matrix()
train_y = np.log(y_train).as_Matrix()

cv_scores = []
kf = model_selection.KFold(n_splits = 5, shuffle = True, random_state = 2017)
for dev_index, val_index in kf.split(range(train_df.shape[0])):
    dev_x, val_x = train_df[dev_index, :], train_df[val_index, :]
    dev_y, val_y = train_y[dev_index], train_y[val_index]
    train_lgb = lgb.Dataset(train_df, train_y)
    model = lgb.train(params, train_lgb, num_boost_round=ROUNDS)
    preds = model.predict(test_df)
    cv_scores.append(explained_variance_score(val_y, preds))
    print(cv_scores)
    break

AttributeError: 'DataFrame' object has no attribute 'as_Matrix'

In [16]:
df_values.describe().T.index

Index([u'full_sq', u'life_sq', u'floor', u'max_floor', u'material',
       u'build_year', u'num_room', u'kitch_sq', u'state', u'area_m',
       ...
       u'big_market_raion', u'nuclear_reactor_raion',
       u'detention_facility_raion', u'water_1line', u'big_road1_1line',
       u'railroad_1line', u'ecology', u'child_on_acc_pre_school',
       u'modern_education_share', u'old_education_build_share'],
      dtype='object', length=408)

In [27]:

import numpy as np
import pandas as pd
import lightgbm as lgb
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV  


RS = 20170629
np.random.seed(RS)

ROUNDS = 450
params = {
	'objective': 'regression',
    'metric': 'rmse',
    'boosting': 'gbdt',
    'learning_rate': 0.04,
    'verbose': 0,
    'num_leaves': 2 ** 5,
    'bagging_fraction': 0.95,
    'bagging_freq': 1,
    'bagging_seed': RS,
    'feature_fraction': 0.7,
    'feature_fraction_seed': RS,
    'max_bin': 100,
    'max_depth': 5,
    'num_rounds': range(400, 500, 20)
}

def tune_model():  
    print "load data ..."  
    d_x = train_df.values  
    d_y = train_y
  
    print "create classifier..."  
    param_grid = {  
        "learning_rate": [i/100.0 for i in range(1,6)],
 
    }  
    params = {  
        'objective': 'regression',
        'metric': 'rmse',
        'boosting': 'gbdt',
        'learning_rate': 0.05,
        'verbose': 0,
        'num_leaves': 2 ** 5,
        'bagging_fraction': 0.95,
        'bagging_freq': 1,
        'bagging_seed': RS,
        'feature_fraction': 0.7,
        'feature_fraction_seed': RS,
        'max_bin': 100,
        'max_depth': 6
    }  
    # max_depth = 7, learning_rate:0.25  
    model = lgb.LGBMRegressor(  
        boosting_type='gbdt', objective="regression")   
    print "run grid search..."  
    searcher = GridSearchCV(estimator=model, param_grid=param_grid, cv=3)  
    searcher.fit(d_x, d_y)  
    print searcher.grid_scores_  
    print "=" * 30, '\n'  
    print searcher.best_params_  
    print "=" * 30, '\n'  
    print searcher.best_score_  
    print "end"
    
tune_model()

load data ...
create classifier...
run grid search...
[mean: 0.02520, std: 0.02874, params: {'learning_rate': 0.01}, mean: 0.07977, std: 0.03004, params: {'learning_rate': 0.02}, mean: 0.12590, std: 0.03127, params: {'learning_rate': 0.03}, mean: 0.16280, std: 0.03350, params: {'learning_rate': 0.04}, mean: 0.19444, std: 0.03577, params: {'learning_rate': 0.05}]

{'learning_rate': 0.05}

0.194444421172
end


c:\python27\lib\site-packages\sklearn\model_selection\_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


In [38]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV  

RS = 20170629
np.random.seed(RS)

ROUNDS = 350
params = { 
        'objective': 'regression',
        'metric': 'rmse',
        'boosting': 'gbdt',
        'learning_rate': 0.05,
        'verbose': 0,
        'num_leaves': 2 ** 5,
        'bagging_fraction': 0.95,
        'bagging_freq': 1,
        'bagging_seed': RS,
        'feature_fraction': 0.7,
        'feature_fraction_seed': RS,
        'max_bin': 100,
        'max_depth': 6
    }  

train_df = df_values[:num_train]
train_y = np.log(y_train)

train_lgb = lgb.Dataset(train_df, train_y)
model = lgb.train(params, train_lgb, num_boost_round=ROUNDS)
preds = model.predict(test_df)


In [40]:
train_df

,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,area_m,...,oil_chemistry_raion,radiation_raion,railroad_terminal_raion,big_market_raion,nuclear_reactor_raion,detention_facility_raion,water_1line,big_road1_1line,railroad_1line,ecology
0,43.0,27.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,6.407578e+06,...,0,0,0,0,0,0,0,0,0,0
1,34.0,19.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,9.589337e+06,...,0,0,0,0,0,0,0,0,0,1
2,43.0,29.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,4.808270e+06,...,0,1,0,0,0,0,0,0,0,2
3,89.0,50.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,1.258354e+07,...,0,0,0,0,0,0,0,0,0,0
4,77.0,77.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,8.398461e+06,...,0,1,1,0,0,0,0,0,1,1
5,67.0,46.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,7.506452e+06,...,0,1,0,0,0,0,0,0,0,2
6,25.0,14.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,1.032047e+07,...,0,0,0,0,0,1,0,0,0,2
7,44.0,44.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,6.407578e+06,...,0,0,0,0,0,0,0,0,0,0
8,42.0,27.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,4.787424e+06,...,0,0,0,0,0,0,0,0,0,2
9,36.0,21.0,9.0,NaN,NaN,NaN,NaN,NaN,NaN,5.235177e+07,...,0,1,0,0,0,0,0,0,0,3


In [39]:
out_df = pd.DataFrame({"id":test_ids, "price_doc":np.exp(preds)})
print(out_df.head(3))

print("Features importance...")
gain = model.feature_importance('gain')
ft = pd.DataFrame({'feature':model.feature_name(), 'split':model.feature_importance('split'), 'gain':100 * gain / gain.sum()}).sort_values('gain', ascending=False)
print(ft.head(25))

plt.figure()
ft[['feature','gain']].head(25).plot(kind='barh', x='feature', y='gain', legend=False, figsize=(10, 20))
plt.gcf().savefig('features_importance.png')

print("Done.")

      id     price_doc
0  30474  5.071425e+06
1  30475  5.362318e+06
2  30476  4.867271e+06
Features importance...
                        feature       gain  split
0                       full_sq  29.992574    408
6                      num_room   4.335966     47
1                       life_sq   3.941998    119
265  cafe_count_5000_price_2500   3.128265     31
267  cafe_count_5000_price_high   1.954186     37
234             cafe_count_3000   1.781144     25
249            sport_count_3000   1.738951     28
386              lifesq_x_state   1.657663    121
211             cafe_count_2000   1.621769     23
370              month_year_cnt   1.290630    198
257             cafe_count_5000   1.115409     30
219  cafe_count_2000_price_2500   0.992349     26
72               metro_min_avto   0.958725     50
5                    build_year   0.908573    131
375                rel_kitch_sq   0.867797    103
377               livArea_ratio   0.840867    193
374                   rel_floor   0

In [31]:
out_df.to_csv("LGB_3.csv", index=False)

In [23]:
former = pd.read_csv('different_result_update2.csv')

In [24]:
former

,id,price_doc
0,30474,5.309800e+06
1,30475,8.126592e+06
2,30476,5.198500e+06
3,30477,5.776796e+06
4,30478,4.961555e+06
5,30479,9.006159e+06
6,30480,4.191041e+06
7,30481,4.136372e+06
8,30482,4.698900e+06
9,30483,4.623848e+06


In [19]:
light = pd.read_csv('lgb_450_20170501.csv')
light

,id,price_doc
0,30474,5.465796e+06
1,30475,7.018418e+06
2,30476,4.859952e+06
3,30477,6.075681e+06
4,30478,5.016307e+06
5,30479,8.236617e+06
6,30480,4.243289e+06
7,30481,3.794256e+06
8,30482,4.842106e+06
9,30483,4.714424e+06


In [35]:
m = former.merge(light, on="id", suffixes=['_louis','_bruno'])

In [36]:
m['price_doc'] = np.exp( .91*np.log(m.price_doc_louis) + .09*np.log(m.price_doc_bruno)) 

In [37]:
m.drop(['price_doc_louis', 'price_doc_bruno'], axis = 1, inplace=True)

In [38]:
m.to_csv('merge_LGB.csv', index=False)

In [39]:
m

,id,price_doc
0,30474,5.340639e+06
1,30475,7.891773e+06
2,30476,5.128954e+06
3,30477,5.835373e+06
4,30478,4.972457e+06
5,30479,8.846703e+06
6,30480,4.201439e+06
7,30481,4.065566e+06
8,30482,4.727198e+06
9,30483,4.641823e+06
